In [15]:
import requests
import pandas as pd
import time

In [16]:
# Steam Storefront API 리뷰 엔드포인트
API_URL = "https://store.steampowered.com/appreviews/1623730"

In [17]:
def fetch_reviews(cursor=""):
    """스토어프론트 API를 사용하여 팰월드의 한국어 리뷰 가져오기"""
    params = {
        "json": 1,
        "language": "koreana",
        "filter": "all",  # 모든 리뷰 가져오기
        "cursor": cursor,
        "store_api_use_in_app": 1,  # **이 옵션이 없으면 0개 리뷰 반환**
    }
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(API_URL, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            reviews = [
                {
                    "review": review["review"],
                    "recommend": review["voted_up"],
                    "votes_up": review["votes_up"],
                    "votes_funny": review["votes_funny"],
                    "timestamp": review["timestamp_created"],
                }
                for review in data.get("reviews", [])
            ]
            next_cursor = data.get("cursor", "")
            return reviews, next_cursor
        else:
            print(f"❌ API 요청 실패: {response.status_code}")
            return [], ""
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")
        return [], ""

In [18]:
if __name__ == '__main__':
    start_time = time.time()
    
    all_reviews = []
    cursor = ""

    while cursor is not None:
        reviews, cursor = fetch_reviews(cursor)
        if reviews:
            all_reviews.extend(reviews)
            time.sleep(1)  # API 요청 간격 조절 (속도 제한 방지)
        else:
            break  # 데이터가 더 이상 없으면 종료

    # DataFrame으로 변환 후 CSV로 저장
    df = pd.DataFrame(all_reviews)
    df.to_csv("palworld_reviews_api.csv", index=False, encoding="utf-8-sig")

    print(f"총 {len(all_reviews)}개의 리뷰를 가져와 'palworld_reviews_api.csv'로 저장했습니다.")
    print(f"⏳ 실행 시간: {time.time() - start_time:.2f}초")

총 0개의 리뷰를 가져와 'palworld_reviews_api.csv'로 저장했습니다.
⏳ 실행 시간: 0.28초
